In [ ]:
#from google.colab import drive
#drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install pyspark
!pip install openclean
!pip install uszipcode
!pip install humanfriendly

     |████████████████████████████████| 281.3 MB 32 kB/s 
     |████████████████████████████████| 198 kB 55.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=fe951d02558394ccfd4ab7a79f9084669a3518fc3f0911d084c111cbe0fdda11
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
     |████████████████████████████████| 267 kB 5.1 MB/s 
     |████████████████████████████████| 137 kB 53.3 MB/s 
     |████████████████████████████████| 260 kB 37.8 MB/s 
     |████████████████████████████████| 69 kB 6.5 MB/s 
     |████████████████████████████████| 109 kB 56.8 MB/s 
     |████████████████████████████████| 525 kB 46.7 MB/s 
     |████████████████████████████████| 206 kB 43.7 MB/s 
     |████████████████████████████████| 596 kB 66.6 MB/s 
     |████████████████████████████████| 180 kB 70.3 MB/s 
     |████████████████████████████████| 149 kB 62.3 MB/s 
     |███████

In [ ]:
import pandas as pd
import uszipcode
import gzip
import humanfriendly
import os

In [ ]:
from openclean.data.source.socrata import Socrata
from openclean.profiling.column import DefaultColumnProfiler
from openclean.profiling.anomalies.sklearn import DBSCANOutliers
from openclean.cluster.key import KeyCollision
from openclean.function.value.key.fingerprint import Fingerprint

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.ml.feature import *

spark = SparkSession.builder.appName("BDProject").getOrCreate()
sc = spark.sparkContext

In [ ]:
df_data = spark.read.format("csv") \
.option("header", "true") \
.option("infer_schema","true")\
.option("first_row_is_header","true")\
.load(r"../data/311_Service_Requests_from_2010_to_Present.csv")
df_data.createOrReplaceTempView("df_data")
df_data.show()

+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+-----------------+---------------+--------------+---------------------+---------------------+------------+----------------+--------+-------------+------+--------------------+----------------------+------------------------------+------------------+----------+---------+--------------------------+--------------------------+----------------------+------------------+------------+------------+--------------------+---------------------+-------------------+------------------------+---------+----------------------+------------------+------------------+--------------------+
|Unique Key|        Created Date|         Closed Date|Agency|         Agency Name|      Complaint Type|          Descriptor|       Location Type|Incident Zip|    Incident Address|      Street Name| Cross Street 1|Cross Street 2|Intersection Street 1|

In [ ]:
df_data.count()

27130961

In [ ]:
df_data = df_data.withColumnRenamed("Unique Key","Unique_Key")
df_data = df_data.withColumnRenamed("Created Date","Created_Date")
df_data = df_data.withColumnRenamed("Closed Date","Closed_Date")
df_data = df_data.withColumnRenamed("Agency Name","Agency_Name")
df_data = df_data.withColumnRenamed("Complaint Type","Complaint_Type")
df_data = df_data.withColumnRenamed("Location Type","Location_Type")
df_data = df_data.withColumnRenamed("Incident Zip","Incident_Zip")
df_data = df_data.withColumnRenamed("Incident Address","Incident_Address")
df_data = df_data.withColumnRenamed("Street Name","Street_Name")
df_data = df_data.withColumnRenamed("Cross Street 1","Cross_Street_1")
df_data = df_data.withColumnRenamed("Cross Street 2","Cross_Street_2")
df_data = df_data.withColumnRenamed("Intersection Street 1","Intersection_Street_1")
df_data = df_data.withColumnRenamed("Intersection Street 2","Intersection_Street_2")
df_data = df_data.withColumnRenamed("Address Type","Address_Type")
df_data = df_data.withColumnRenamed("Facility Type","Facility_Type")
df_data = df_data.withColumnRenamed("Due Date","Due_Date")
df_data = df_data.withColumnRenamed("Resolution Description","Resolution_Description")
df_data = df_data.withColumnRenamed("Resolution Action Updated Date","Resolution_Action_Updated_Date")
df_data = df_data.withColumnRenamed("Community Board","Community_Board")
df_data = df_data.withColumnRenamed("Park Borough","Park_Borough")

In [ ]:
df_data.show()

+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+-----------------+---------------+--------------+---------------------+---------------------+------------+----------------+--------+-------------+------+--------------------+----------------------+------------------------------+------------------+----------+---------+--------------------------+--------------------------+----------------------+------------------+------------+------------+--------------------+---------------------+-------------------+------------------------+---------+----------------------+------------------+------------------+--------------------+
|Unique_Key|        Created_Date|         Closed_Date|Agency|         Agency_Name|      Complaint_Type|          Descriptor|       Location_Type|Incident_Zip|    Incident_Address|      Street_Name| Cross_Street_1|Cross_Street_2|Intersection_Street_1|

##Removing Duplicate value rows for Unique Key Column

We first check if Unique Key column has any Duplicate Data

In [ ]:
df1=df_data.groupBy("Unique_Key").count().filter("count > 1")
df1.drop('count').show()

+----------+
|Unique_Key|
+----------+
+----------+



If the column has instances of Duplicate data, we can remove the duplicate data rows usng the below script

In [ ]:
df_data = df_data.dropDuplicates(['Unique_Key'])

## Date Conversion

In [ ]:
from pyspark.sql.functions import to_timestamp

Checking the type of Date column.

In [ ]:
df_data.select("Created_Date").dtypes

[('Created_Date', 'string')]

If datatype found to anything other than timestamp, we convert the column to timestamp using the format as follows: "MM/dd/yyyy hh:mm:ss a"

In [ ]:
df_data=df_data.withColumn('Created_Date',to_timestamp(df_data.Created_Date,"MM/dd/yyyy hh:mm:ss a"))
df_data=df_data.withColumn('Closed_Date',to_timestamp(df_data.Closed_Date,"MM/dd/yyyy hh:mm:ss a"))
df_data=df_data.withColumn('Due_Date',to_timestamp(df_data.Closed_Date,"MM/dd/yyyy hh:mm:ss a"))

As seen below, the types for selected columns have been changed to Timestamp

In [ ]:
df_data.printSchema()

root
 |-- Unique_Key: string (nullable = true)
 |-- Created_Date: timestamp (nullable = true)
 |-- Closed_Date: timestamp (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Agency_Name: string (nullable = true)
 |-- Complaint_Type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- Location_Type: string (nullable = true)
 |-- Incident_Zip: string (nullable = true)
 |-- Incident_Address: string (nullable = true)
 |-- Street_Name: string (nullable = true)
 |-- Cross_Street_1: string (nullable = true)
 |-- Cross_Street_2: string (nullable = true)
 |-- Intersection_Street_1: string (nullable = true)
 |-- Intersection_Street_2: string (nullable = true)
 |-- Address_Type: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Landmark: string (nullable = true)
 |-- Facility_Type: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Due_Date: timestamp (nullable = true)
 |-- Resolution_Description: string (nullable = true)
 |-- Resoluti

## Check Mapping of Created Date to CLosed Date.

Closed Date should always be bigger than Created Date.

In [ ]:
df_new = df_data.filter(df_data.Closed_Date > df_data.Created_Date)

Getting count of rows having valid Created Date < Closed Date values

In [ ]:
df_new.count()

25117976

Checking for empty/Null Closed Date values. The rows with these values are valid to the dataset since these rows represent Requests in In-Progress status.

In [ ]:
df_new2 = df_data.filter(df_data.Closed_Date.isNull())

In [ ]:
df_new2.count()

756719

In [ ]:
df_new2.show()

+----------+-------------------+-----------+------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+---------------+------------------+---------------------+---------------------+------------+-------------------+--------+-------------+--------+--------+----------------------+------------------------------+---------------+----------+---------+--------------------------+--------------------------+----------------------+--------------------+------------+------------+--------------------+---------------------+-------------------+------------------------+---------+----------------------+------------------+------------------+--------------------+
|Unique_Key|       Created_Date|Closed_Date|Agency|         Agency_Name|      Complaint_Type|          Descriptor|       Location_Type|Incident_Zip|    Incident_Address|         Street_Name| Cross_Street_1|    Cross_Street_2|Intersection_Street_1|Intersection_S

We check the values for Closed Date < Created Date and later discard the corresponding rows from our data.

In [ ]:
df_new3 = df_data.filter(df_data.Closed_Date < df_data.Created_Date)

In [ ]:
df_new3.count()

268151

In [ ]:
df_new3.show()

+----------+-------------------+-------------------+------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+-------------------+--------------------+---------------------+---------------------+------------+-------------+--------+-------------+--------+-------------------+----------------------+------------------------------+--------------------+----------+-------------+--------------------------+--------------------------+----------------------+------------------+-------------+------------+--------------------+---------------------+-------------------+------------------------+---------+----------------------+------------------+------------------+--------------------+
|Unique_Key|       Created_Date|        Closed_Date|Agency|         Agency_Name|      Complaint_Type|          Descriptor|       Location_Type|Incident_Zip|    Incident_Address|         Street_Name|     Cross_Street_1|      Cross_Stre

Combining rows Containing "Created Date < Closed Date" and "Closed Date = Null"

In [ ]:
from functools import reduce 

In [ ]:
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [ ]:
df_data2 = unionAll(*[df_new,df_new2])

In [ ]:
df_data2.count()

25874695

In [ ]:
df_data2.show()

+----------+-------------------+-------------------+------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+----------------+------------------+---------------------+---------------------+------------+--------+--------+-------------+------+-------------------+----------------------+------------------------------+---------------+----------+-----------+--------------------------+--------------------------+----------------------+------------------+------------+------------+--------------------+---------------------+-------------------+------------------------+---------+----------------------+------------------+------------------+--------------------+
|Unique_Key|       Created_Date|        Closed_Date|Agency|         Agency_Name|      Complaint_Type|          Descriptor|       Location_Type|Incident_Zip|    Incident_Address|         Street_Name|  Cross_Street_1|    Cross_Street_2|Intersection_Street_